In [0]:
def model_evaluation(result_path):
  result = pd.read_csv(result_path)
  result = result.drop(columns = ['Unnamed: 0'])
  result['DATE'] = pd.to_datetime(result['DATE'])
  Asset_list = pd.read_csv("/dbfs/FileStore/shared_uploads/asingha3@networkrail.co.uk/RADAR_Total_Asset_List.csv")
  Asset_list = Asset_list[["EllipseNo","AssetName"]]
  Asset_list["EllipseNo"] = Asset_list["EllipseNo"].astype("str").str.pad(width=12, side='left', fillchar='0')
  Asset_list.rename(columns = {'EllipseNo':"EllipseID"},inplace=True)
  Asset_list.rename(columns = {'AssetName':"Name"},inplace=True)
  result = result.merge(Asset_list,how='left',on=['Name'])
  Loggers_list = result.EllipseID.unique().tolist()
  
  FMS_Faults_clamplockph3= FMS_Faults = spark.read.jdbc(url=innvUrl, table='dbo.tbl_cf10_AssetFMS_InDT', properties=properties)
  FMS_Faults_clamplockph3 = FMS_Faults_clamplockph3.filter(FMS_Faults_clamplockph3.EQUIP_NO.isin(Loggers_list))
  FMS_Faults_clamplockph3 = FMS_Faults_clamplockph3.select(['EQUIP_NO', 'COMPONENT_LEVEL_1','FAILED_DT','FAULT_RESOLVED_DT','InOrderDate','COMPONENT_LEVEL_2','FAILURE_NUMBER','D_FAILURE_KEY'])
  FMS_Faults_clamplockph3 = FMS_Faults_clamplockph3.withColumn('InOrderDate' , fn.to_date(fn.col('InOrderDate'),format ="YYYY-MM-ddTHH:MM:SS.sss+ssss"))
  FMS_Faults_clamplockph3 = FMS_Faults_clamplockph3.withColumn('FAILED_DT',fn.to_date(fn.col('FAILED_DT'),format ="YYYY-MM-ddTHH:MM:SS.sss+ssss"))
  FMS_Faults_clamplockph3 = FMS_Faults_clamplockph3.withColumn('FAULT_RESOLVED_DT',fn.to_date(fn.col('FAULT_RESOLVED_DT'),format ="YYYY-MM-ddTHH:MM:SS.sss+ssss"))
  
  WO = spark.read.jdbc(url=jdbcUrl, table='DST.v_cf10_AssetWorkorder', properties=connectionProperties)

  WO = WO.filter(WO.equip_no.isin(Loggers_list))

  #display(WO)

  #WO = WO.filter(WO.EQUIP_NO.isin(loggers_list)) 

  Work_Order_clamplockph3 =WO.select('equip_no','WO_CLOSED_DATE','WORK_ORDER_STATUS','WORK_ORDER_DESC')

  Work_Order_clamplockph3 = Work_Order_clamplockph3[Work_Order_clamplockph3['WORK_ORDER_STATUS']=='Closed']

  Work_Order_clamplockph3 =Work_Order_clamplockph3[['equip_no','WO_CLOSED_DATE','WORK_ORDER_DESC']]

  old_column = ['equip_no','WO_CLOSED_DATE','WORK_ORDER_DESC']

  newcolumns =['EQUIP_NO','WO_DATE','WORK_ORDER_DESC']

  for i in range(0,2):
    Work_Order_clamplockph3 = Work_Order_clamplockph3.withColumnRenamed(old_column[i],newcolumns[i])
    
  Faults = FMS_Faults_clamplockph3.toPandas()
  WO = Work_Order_clamplockph3.toPandas()
  WO = WO.rename(columns = {"WO_DATE":"WO_CLOSED_DATE"})
  WO = WO.rename(columns = {"EQUIP_NO":"EllipseID"})
  WO["Impact_on_POE"] = 1
  Faults = Faults.rename(columns = {"EQUIP_NO":"EllipseID"})
  
  Result_sdf = spark.createDataFrame(result)
  Faults['FAILED_DT'] = pd.to_datetime(Faults['FAILED_DT'])
  Faults['FAULT_RESOLVED_DT'] = pd.to_datetime(Faults['FAULT_RESOLVED_DT'])

  Faults['InOrderDate'] = pd.to_datetime(Faults['InOrderDate'])

  WO['WO_CLOSED_DATE'] = pd.to_datetime(WO['WO_CLOSED_DATE'])
  
  Result_sdf = Result_sdf.withColumn('DATE',fn.to_date(fn.col('DATE'),format="yyy-MM-DDTHH:MM:SS.sss+sss"))

  Result_sdf = Result_sdf.dropDuplicates()

  new_WO = WO.groupby(["EllipseID","WO_CLOSED_DATE"], as_index = False)["Impact_on_POE"].sum()

  new_WO = new_WO[new_WO["Impact_on_POE"] > 0]

  new_WO["EllipseID"] = new_WO["EllipseID"].astype("str").str.pad(width=12, side='left', fillchar='0')

  new_WO.rename(columns = {'EllipseID':'EQUIP_NO'},inplace=True)

  Faults["EllipseID"] = Faults["EllipseID"].astype("str").str.pad(width=12, side='left', fillchar='0')

  new_FMS = Faults.copy()

  new_FMS.rename(columns ={"EllipseID":"EQUIP_NO"},inplace = True)

  new_WO = spark.createDataFrame(new_WO)

  new_FMS = spark.createDataFrame(new_FMS)

  new_WO = new_WO.withColumn('WO_CLOSED_DATE',fn.to_date(fn.col('WO_CLOSED_DATE')))

  new_FMS = new_FMS.withColumn('FAILED_DT',fn.to_date(fn.col('FAILED_DT')))

  new_FMS = new_FMS.withColumn('FAULT_RESOLVED_DT',fn.to_date(fn.col('FAULT_RESOLVED_DT')))

  new_FMS = new_FMS.withColumn('InOrderDate',fn.to_date(fn.col('InOrderDate')))

  work_order = new_WO.withColumn('prev_dt_WO',fn.lag(fn.col('WO_CLOSED_DATE')).over(Window.partitionBy('EQUIP_NO').orderBy('WO_CLOSED_DATE')))

  default_date= '2017-01-01'

  work_order =work_order.withColumn('prev_dt_WO',fn.coalesce(fn.col('prev_dt_WO'),fn.lit(default_date)))

  work_order=work_order.dropDuplicates(['equip_no','WO_CLOSED_DATE'])

  Result_WO = Result_sdf.join(work_order,(Result_sdf['EllipseID']==work_order['EQUIP_NO'])&(Result_sdf['DATE']>work_order['prev_dt_WO'])&(Result_sdf['DATE']<=work_order['WO_CLOSED_DATE']),how='left')

  Result_WO = Result_WO.withColumn('Day_before_WO',fn.datediff('WO_CLOSED_DATE','DATE'))

  faults = new_FMS

  faults = faults.withColumn('prev_dt_fault',fn.lag(fn.col('FAILED_DT')).over(Window.partitionBy('EQUIP_NO').orderBy('FAILED_DT')))

  default_date= '2017-01-01'

  faults =faults.withColumn('prev_dt_fault',fn.coalesce(fn.col('prev_dt_fault'),fn.lit(default_date)))

  faults=faults.dropDuplicates(['EQUIP_NO','FAILED_DT'])

  result_fault_WO = Result_WO.join(faults,(Result_WO['EllipseID']==faults['EQUIP_NO'])&(Result_WO['DATE']>faults['prev_dt_fault'])&(Result_WO['DATE']<=faults['FAILED_DT']),how='left')

  result_fault_WO = result_fault_WO.withColumn('Day_before_fault',fn.datediff('FAILED_DT','DATE'))

  equip_date = result_fault_WO.select('EllipseID','DATE')

  faults = faults.withColumn('next_fault_dt',fn.lag(fn.col('FAILED_DT'),-1).over(Window.partitionBy('EQUIP_NO').orderBy('FAILED_DT')))

  resolving_day1 = equip_date.join(faults,(equip_date['EllipseID']==faults['EQUIP_NO'])&(equip_date['DATE']>faults['FAILED_DT'])&(equip_date['DATE']<=faults['InOrderDate'])&(equip_date['DATE']<faults['next_fault_dt']),how='left')

  resolving_day = resolving_day1.withColumn('resolving_day',fn.when(fn.col('InOrderDate').isNotNull(),fn.lit('y')).otherwise('n'))

  resolving_day =resolving_day.select('EllipseID','DATE','resolving_day')

  result_fault_WO = result_fault_WO.sort('EllipseID','DATE')

  feat_all = result_fault_WO.join(resolving_day,['EllipseID','DATE'],how='left')

  feat_all =feat_all.drop('EQUIP_NO','prev_dt')

  feat_all = feat_all.dropDuplicates()

  Result_sdf= feat_all

  Result_df = Result_sdf.toPandas()
  
  Results = Result_df.copy()
  
  Results['Day_before_WO'] = Results['Day_before_WO'].fillna(999)

  Results['Day_before_fault'] = Results['Day_before_fault'].fillna(999)

  Results_TP = Results.loc[(Results['Day_before_WO']>0)&(Results['Day_before_fault']>0)&(Results['Day_before_fault']<8)]


  Total_failures = Results_TP.FAILURE_NUMBER.nunique()
  #can be changed

  TP_num = Results_TP.loc[Results_TP['final_prediction']==0].FAILURE_NUMBER.nunique()

  TP = (TP_num/Total_failures)*100

  Results_FP = Results.loc[(Results['Day_before_WO']>=8)&(Results['Day_before_fault']>=8)]


  Results_FP_den = Results_FP.shape[0]

  Results_FP_num = Results_FP.loc[Results_FP['final_prediction']==0].shape[0]

  FP = (Results_FP_num/Results_FP_den)*100

  return(print("TP {}/{}->{:.2f}%   FP {}/{}->{:.2f}%".format(TP_num,Total_failures,TP,Results_FP_num,Results_FP_den,FP)))

In [0]:
model_evaluation(result_path)

/databricks/spark/python/pyspark/sql/pandas/utils.py:81: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [D_FAILURE_KEY] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 "conversion.".format(", ".join(decimal_col_names))
TP 21/43->48.84% FP 497/5083->9.78%